# Read Keystroke Data and Build Model to Process 

In [10]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import datasets

input_path = "Dataset/Kevin and Maxion/DSL-StrongPasswordData.csv"
print input_path;

Dataset/Kevin and Maxion/DSL-StrongPasswordData.csv


Import Data and Extract Features

In [24]:
#!ls
# rank1_array = np.array([1, 2, 3]);
# print("type of rank1_array:", type(rank1_array));
# print("shape of rank1_array:", rank1_array.shape);
# print("elements in rank1_array:", rank1_array[0], rank1_array[1], rank1_array[2]);

# plt.plot([1,2,3,4]);
# plt.ylabel('custom y label');
# plt.show();

file = open(input_path, "r");

data = file.read();
print type(data);
onelist = data.splitlines();
data_lines = np.array(data.splitlines());
print type(onelist);

print type(data_lines);
print data_lines.size;

<type 'str'>
<type 'list'>


AttributeError: 'list' object has no attribute 'size'